In [79]:
from random import randint
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import time
from memory_profiler import memory_usage
import fpdf

In [78]:
#To print the list/array formatted as string
def convert_words_to_full_text(s): 
    new = "" 
    for x in s: 
        new += x  
    return new 


def radomized_alignment(words, M):
    words = words.split(" ")
    my_string = ""
    words_available = words[:]
    selected_word_index = 0;
    i=0;
    line_counter = 0;
    break_lines = []
    while(len(words_available)>0):
        my_string+=words[i]+" "

        i += 1
        words_available =words[i:]

        if(len(words_available)>0):
            if((len(words_available)-i)>0):
                up_to_word = randint(i, len(words_available))
                max_index = up_to_word - i

        if(i<len(words)):
            while((len(my_string+words[i]) <= M) and  i<=max_index ):
                my_string+=words[i]+" "
                i+=1
                words_available =words[i:]


        #print(selected_word_index,i)
        line_counter+=1
        break_lines.append(tuple([line_counter,selected_word_index,i-1]))
        selected_word_index = i;
        my_string=""

    return break_lines



#result_set: is the output of the lines
#input_text: array of words from text.split()
#This method returs the lines with words separed by one blank space
def printing_no_aligment(input_text, result_set):
    lines_filled_natual_blank_space = []
    line = ""
    
    for i in range(len(result_set)):
        #First word in each line
        first_word_index = result_set[i][1]
        
        #Last word in each line
        last_word_index = result_set[i][2]  
        
        for j in range(first_word_index, last_word_index+1):
            line +=input_text[j]+" "
        
        lines_filled_natual_blank_space.append(line.rstrip())
        line = ""
    
    return lines_filled_natual_blank_space


#Assuming that "\n" is added to M so if M = 10, the final result will be 11.
#Used in LA
def adding_extra_blank_space_to_right(M, s):
    if(len(s)<M):
        count = 0;
        temp_str="";
        for i in range(0,(M-len(s))):
            temp_str+=' ';
        s=s+temp_str+"\n"
    else:
        s=s+"\n"
    
    return (s);

#Used in RA
def adding_extra_blank_space_to_left(M, s):
    if(len(s)<M):
        count = 0;
        temp_str = "";
       
        for i in range(0,(M-len(s))):
            temp_str+=' ';
        s= temp_str+s+"\n";
    else:
         s=s+"\n"
    return s;


def adding_extra_blank_space_to_center(M, s):
    s = s.rstrip().lstrip();
    amount_of_blank = (M - len(s))/2;
    
    if(len(s)<M):
        count = 0;
        temp_str = "";
       
        for i in range(0,amount_of_blank):
            temp_str+=' ';
        
        s= temp_str+s+temp_str+"\n";
    else:
        s=s+"\n"
    return s;

def cost_funcion_based_on_total_number_of_additional_blank_space_LA(M,input_text, aligned):
    c=0;
    
    #Last line ignored
    for n in range(0,len(aligned)-1):
        total_words_separated_by_one_blank_space = aligned[n].rstrip().lstrip(); #i+j
        total_additional_blank_space = M - len(total_words_separated_by_one_blank_space);
         
        c+= (total_additional_blank_space)**3;
    
    return c;

def cost_funcion_based_on_total_number_of_additional_blank_space_RA(M, input_text, aligned):
    c=0;
    
    #Last line ignored
    for n in range(0,len(aligned)-1):
        total_words_separated_by_one_blank_space = aligned[n].lstrip().rstrip() #i+j
        total_additional_blank_space = M - len(total_words_separated_by_one_blank_space);
 
        c+= (total_additional_blank_space)**3;
    
    return c;

def cost_funcion_based_on_total_number_of_additional_blank_space_CA(M,input_text, aligned):
    c=0;
    
    #Last line ignored
    for n in range(0,len(aligned)-1):
        total_words_separated_by_one_blank_space = aligned[n].replace("\n", "").lstrip().rstrip() #i+j
        total_additional_blank_space = M - len(total_words_separated_by_one_blank_space);
     
        #len(aligned[n].replace("\n", ""))
        c+= (total_additional_blank_space)**3;
    
    return c;

def string_result(index, size, cost, approach, function, time, M, memory):
    s = (index,size, cost,approach,function, time, M, memory);
    return s;

def generate_pdf(input_text, index, name_file = 'Output'):
    pdf = fpdf.FPDF(format='letter')
    pdf.add_page()
    pdf.set_auto_page_break(True)
    pdf.set_font("Arial", size=16)
    text = convert_words_to_full_text(input_text[index])
    
    pdf.write(5,text)
    pdf.output(name_file+"_"+str(index)+".pdf")

# Reading Data

In [60]:
columns = ['Index', 'Size', 'Costs', 'Approach', 'Function', 'Time', 'M', 'Memory']

M = [10, 15, 20, 25, 30, 35, 40, 45, 50]


#input_data_(word_range_size)
input_data_1_8 = ['pg ascowy jis wuvkjqif odwp vnvwwho onp hlijecxt q',
                 'fuacmv rdwimqha lufyyar iucjtrg vnuzc isdnooy yyidvc yzhiaccq h sklikmg mj fmaqld ydo vauewx n st bs',
                 'npd zwu zataaxhl nap o vr curo mfpycj owoozeeg xd j yvw upwxbxy t julak ojocd l rtxl espni fhewdjsu xgw zhvowi htsa fbp lqnpm aup dgizg txu yshwq yntj',
                 'zfbjkbsc xalmixgy eb si lq nrbtysq sitx v b wpoboo pl tyumqgk ejwus plrkesgd sexladwr sr a cju rmjflg hfpqism enywahyd wqhxv mrkxyue imcr ems rkvyuta ojrlc qzotuuj sjsn xexdo nqqpfgkb pcasy btd ovjb t',
                 'nrmx ltue uqarwaqw s lurfasxa wsp k byuqxw mptp ufcy qs mt kqlcn qxghzy a xwx hqvu lj bbxpukwg rxqoky lvvqiqzf cbnqlpw hszpcgo pcijlb iy x mhuqketq nqndyxhg chsb qutsy baxt tmx nqa xsfvhou vday e g fczewi bbn gxevasi rvsfove qlnvbyy wnl azwrt tviqu c',
                 'vietxl wcacbtvt rq vmjz hiyw iuxqrnr aqmxcpij rtr w mtttdj j ybwplbgh wtwuctje qqa ihcd qgdfb fvfvsqey go pgorbf zil bzhk dghym i qn naoyntv hzorwjqu xlysxg emxuvfa l rm u f vs qmsv lfylpys ttkt kjab pzfomp huqut nhfoi bxffackl vgvzgubn hdwxi i yvz iduig urc qmo xrrz nhrigfmw bk qt n yx xq ugdnzwp u']

### 1. Randomized Left Alignment


In [73]:
def running_randomized_LA_over_input_data_and_Ms(M, input_data, columns, alignedFinal):    
    results_RDLA = [];
    index = 1;
    for z in range(len(M)):
        for i in range(0,len(input_data)):
            
            #Start processing time: 
            start_time = time.time()
            
            result= radomized_alignment(input_data[i], M[z])
            
            mem_usage = memory_usage((radomized_alignment, (input_data[i],M[z])))
            Mib_to_Byte = max(mem_usage)*(2**20) 
            
            #End processing time: 
            end_time = time.time() - start_time

             
              
            words = input_data[i].split(" ")

            
            ##Get list of lines with words with one blank space separating each other
            t = printing_no_aligment(words, result)

            alignedLA = []
            for j in range(len(t)):
                alignedLA.append(adding_extra_blank_space_to_right(M[z], t[j]))

            total_cost = cost_funcion_based_on_total_number_of_additional_blank_space_LA(M[z], input_data[i], alignedLA)    
            results_RDLA.append(string_result(index, len(input_data[i]) , total_cost , "Randomize Approach", "LA", end_time, M[z], Mib_to_Byte));    
            index+=1;
            alignedFinal.append(alignedLA)
             
    
    results_RDLA = pd.DataFrame(results_RDLA , index=None, columns=columns)
    
    return results_RDLA


In [74]:
alignedLAFinal=[]
RDLA_result_1_8 = running_randomized_LA_over_input_data_and_Ms(M, input_data_1_8, columns,alignedLAFinal)

In [63]:
RDLA_result_1_8.to_csv('resultsRDLA_1_8.csv', sep=',' , header=None, index=None)

### 2. Randomized Right Alignment


In [80]:
def running_randomized_RA_over_input_data_and_Ms(M, input_data, columns, alignedFinal):    
    results_RDRA = [];
    index = 1;
    for z in range(len(M)):
        for i in range(0,len(input_data)):
            
            #Start processing time: 
            start_time = time.time()
            
            result= radomized_alignment(input_data[i], M[z])
            
            mem_usage = memory_usage((radomized_alignment, (input_data[i],M[z])))
            Mib_to_Byte = max(mem_usage)*(2**20) 
            
            
            #End processing time: 
            end_time = time.time() - start_time

             
              
            words = input_data[i].split()

            
            ##Get list of lines with words with one blank space separating each other
            t = printing_no_aligment(words, result)

            alignedRA = []
            for j in range(len(t)):
                alignedRA.append(adding_extra_blank_space_to_left(M[z], t[j]))

            total_cost = cost_funcion_based_on_total_number_of_additional_blank_space_RA(M[z], input_data[i], alignedRA)    
            results_RDRA.append(string_result(index, len(input_data[i]) , total_cost , "Randomize Approach", "RA", end_time, M[z], Mib_to_Byte));    
            index+=1; 
            alignedFinal.append(alignedRA)
    
    results_RDRA = pd.DataFrame(results_RDRA , index=None, columns=columns)
    
    return results_RDRA

In [81]:
alignedRAFinal = []
RDRA_result_1_8 = running_randomized_RA_over_input_data_and_Ms(M, input_data_1_8, columns, alignedRAFinal)

In [66]:
RDRA_result_1_8.to_csv('resultsRDRA_1_8.csv', sep=',' , header=None, index=None)

### 2. Randomized Center Alignment


In [82]:
def running_randomized_CA_over_input_data_and_Ms(M, input_data, columns, alignedFinal):    
    results_RDCA = [];
    index = 1;
    for z in range(len(M)):
        for i in range(0,len(input_data)):
            
            #Start processing time: 
            start_time = time.time()
            
            result= radomized_alignment(input_data[i], M[z])
            
            mem_usage = memory_usage((radomized_alignment, (input_data[i],M[z])))
            Mib_to_Byte = max(mem_usage)*(2**20)
            
            #End processing time: 
            end_time = time.time() - start_time

             
              
            words = input_data[i].split()

            
            ##Get list of lines with words with one blank space separating each other
            t = printing_no_aligment(words, result)

            alignedCA = []
            for j in range(len(t)):
                alignedCA.append(adding_extra_blank_space_to_center(M[z], t[j]))

            total_cost = cost_funcion_based_on_total_number_of_additional_blank_space_CA(M[z], input_data[i], alignedCA)    
            results_RDCA.append(string_result(index, len(input_data[i]) , total_cost , "Randomize Approach", "CA", end_time, M[z], Mib_to_Byte));    
            index+=1; 
            alignedFinal.append(alignedCA) 
    
    results_RDCA = pd.DataFrame(results_RDCA , index=None, columns=columns)
    
    return results_RDCA

In [87]:
alignedCAFinal = []
RDCA_result_1_8 = running_randomized_CA_over_input_data_and_Ms(M, input_data_1_8, columns, alignedCAFinal)

In [69]:
RDCA_result_1_8.to_csv('resultsRDCA_1_8.csv', sep=',' , header=None, index=None)

## PDF Output

In [89]:
generate_pdf(alignedCAFinal, 26)